# ERA5 Resolution Improvement
---
This notebook is concerned with the improvement of the grid resolution of the time series forecast results. Each one of the 3,195 grids must be split into 10,000 (100 x 100) which corresponds to the resolution of the GWA map.

This notebook requires the following:
* the **resulting pickle file from NB # 1**, which could not be made available through Github due to filesize
* the **Global Wind Atlas map** used as input in NB # 1, available in the Github repository
* the **resulting pickle files from NB # 6**, available in the Github repository

In [1]:
import numpy as np
import pandas as pd
import rasterio as rio
import math
from datetime import datetime

In [2]:
gwa_df = pd.read_pickle("results/gwa0817_100m.pkl")
gwa_df

,x,y,windvelo
0,0,5801,5.230973
1,0,5802,5.231110
2,0,5803,5.231216
3,0,5804,5.231292
4,0,5805,5.232414
...,...,...,...
22423447,4950,5957,5.187756
22423448,4950,5958,5.187970
22423449,4950,5959,5.188166
22423450,4950,5960,5.188385


In [3]:
tif_file_name = "data/PHL_wind-speed_100m.tif"
with rio.open(tif_file_name) as img:
    affine_matrix = img.transform
    inverse_affine = ~affine_matrix
img.close()

In [4]:
print(affine_matrix) # longitude, latitude = affine_matrix * (x, y)
print(inverse_affine) # x, y = inverse_affine * (longitude, latitude)

| 0.00, 0.00, 116.04|
| 0.00,-0.00, 22.18|
| 0.00, 0.00, 1.00|
| 400.00, 0.00,-46416.63|
| 0.00,-400.00, 8871.31|
| 0.00, 0.00, 1.00|


In [11]:
x_list = gwa_df.x.unique()
len(x_list)

4951

In [12]:
y_list = gwa_df.y.unique()
len(y_list)

7662

In [17]:
gwa_df["lat"] = pd.Series(dtype="float32")
gwa_df["lon"] = pd.Series(dtype="float32")
gwa_df

,x,y,windvelo,lat,lon
0,0,5801,5.230973,NaN,NaN
1,0,5802,5.231110,NaN,NaN
2,0,5803,5.231216,NaN,NaN
3,0,5804,5.231292,NaN,NaN
4,0,5805,5.232414,NaN,NaN
...,...,...,...,...,...
22423447,4950,5957,5.187756,NaN,NaN
22423448,4950,5958,5.187970,NaN,NaN
22423449,4950,5959,5.188166,NaN,NaN
22423450,4950,5960,5.188385,NaN,NaN


## ! ! ! Warning ! ! !
The following 2 cells may run for at least 45 minutes each.

In [29]:
for i in range(len(x_list)):
    x = x_list[i]
    row_index = gwa_df.x == x
    gwa_df.loc[row_index, "lon"] = gwa_df[row_index].apply(lambda g: (affine_matrix * (g.x, 0))[0], axis=1)

In [30]:
for i in range(len(y_list)):
    y = y_list[i]
    row_index = gwa_df.y == y
    gwa_df.loc[row_index, "lat"] = gwa_df[row_index].apply(lambda g: (affine_matrix * (0, g.y))[1], axis=1)

In [36]:
gwa_df = gwa_df.astype({"lon": "float32"})

In [40]:
gwa_df.dtypes

x             int32
y             int32
windvelo    float32
lat         float32
lon         float32
dtype: object

In [39]:
gwa_df.describe()

,x,y,windvelo,lat,lon
count,2.242345e+07,2.242345e+07,2.242345e+07,2.242345e+07,2.242345e+07
mean,2.461363e+03,4.170244e+03,5.985596e+00,1.197119e+01,9.737663e+01
std,1.074798e+03,1.803497e+03,1.743064e+00,4.410810e+00,2.493506e+01
min,0.000000e+00,9.000000e+00,5.363950e-01,3.003269e+00,1.160416e+02
25%,1.615000e+03,2.810000e+03,4.338831e+00,8.115768e+00,1.200791e+02
50%,2.406000e+03,4.384000e+03,5.492495e+00,1.121827e+01,1.220566e+02
75%,3.221000e+03,5.625000e+03,6.777289e+00,1.515327e+01,1.240941e+02
max,4.950000e+03,7.670000e+03,1.763615e+01,2.215577e+01,1.284166e+02


In [41]:
gwa_df.to_pickle("results/gwa_grids.pkl")

In [42]:
def get_square_bounds_ne(ctr_lat, ctr_lon, side_len=0.25):
    adjust = side_len / 2
    top_left = [ ctr_lat + adjust, ctr_lon - adjust ]
    top_right = [ ctr_lat + adjust, ctr_lon + adjust ]
    bottom_left = [ ctr_lat - adjust, ctr_lon - adjust ]
    bottom_right = [ ctr_lat - adjust, ctr_lon + adjust ]
    return [ top_left, bottom_right ]

In [43]:
mean2018_df = pd.read_pickle("results/mean2018.pkl")
mean2019_df = pd.read_pickle("results/mean2019.pkl")
mean2020_df = pd.read_pickle("results/mean2020.pkl")
mean2018_df.shape, mean2019_df.shape, mean2020_df.shape

((3195, 17), (3195, 17), (3195, 17))

In [44]:
mean2018_df["bounds"] = mean2018_df.apply(lambda x : get_square_bounds_ne(x["lat"], x["lon"]), axis=1)
mean2019_df["bounds"] = mean2019_df.apply(lambda x : get_square_bounds_ne(x["lat"], x["lon"]), axis=1)
mean2020_df["bounds"] = mean2020_df.apply(lambda x : get_square_bounds_ne(x["lat"], x["lon"]), axis=1)
mean2018_df.shape, mean2019_df.shape, mean2020_df.shape

((3195, 18), (3195, 18), (3195, 18))

In [45]:
mean2018_df.to_pickle("results/mean2018.pkl")
mean2019_df.to_pickle("results/mean2019.pkl")
mean2020_df.to_pickle("results/mean2020.pkl")